# The static optimization
One of the least properly named analysis in biomechanics, the static optimization is often the last step in the inverse flow in biomechanics (that is starting from the data and going back to the cause, which are the muscle activations). 
So it is basically to find which muscle activation set can explain some values found in inverse dynamics. 

As there are infinitely many solutions that can explain the forces, unless they want to find the answer by hand, one must perform an optimization. 
Since there is an optimization at each frame, the dynamic of the system is basically ignore (therefore the `static` in static optimization). 

In this tutorial, we will use `scipy` to perform an optimization at each frame to find the optimal set of muscle activation

## Load a previously created bioMod file and the data of inverse kinematic and dynamics
This first section prepares Python and load a `.bioMod` file as shown in the `2-dynamic_model_creation` or `2.5-simple_upper_limb` script with the data generated by `1-inverse_kinematics` and `3-inverse_dynamics`.

In [ ]:
# So first, let's import all the required modules 
from biorbd import Biorbd  # This is the core that will do all the calculation
from scipy import optimize

from copy import copy  # Allows to copy arrays
import numpy  # Numpy is a python module that helps dealing with the mathematics of matrices and vectors
numpy.set_printoptions(precision=4, suppress=True)  # Make the printing of numpy matrices prettier

from matplotlib import pyplot

In [ ]:
# Import and interpret the bioMod of the full body (created by the script `2-dynamic_model_creation` or `2.5-simple_upper_limb`)
# Please note that the dynamic model must be loaded
use_upper_limb = False  # So use the upper limb model or the full body

if use_upper_limb:
    model = Biorbd("models/SimpleUpperLimbWithMuscles.bioMod")
    all_q, all_qdot, all_qddot, all_tau = numpy.load("results/inverse_dynamics_upper_body.npy")
else:
    model = Biorbd("models/SimpleBodyWithMuscles.bioMod")
    all_q, all_qdot, all_qddot, all_tau = numpy.load("results/inverse_dynamics.npy")

# Get the vector time and some useful variables
time_vector = numpy.load("results/time_vector.npy")
n_frames = all_q.shape[1]

n_q = model.nb_q
n_muscles = len(model.muscles)
muscle_names = [m.name for m in model.muscles]
dof_names = model.dof_names
rested_position = all_q[:, 0]

## The static optimization
Static optimization in a nutshell is an optimization that finds the minimized activation of the muscles that matches the dynamics of a system.
So now let's define the optimization.

In [ ]:
# So let's first define a function that converts muscle activation to joint torque
def muscular_joint_torque(activations, q, qdot):
    return model.muscles.joint_torque(activations=activations, q=q, qdot=qdot)


# Let's test our function with biceps fully activated (and triceps deactivated)
q = numpy.array(rested_position)
qdot = numpy.zeros(n_q)
act = (0.99999, 0.00001)  # Please note this will only work for a two muscles model
print(f"The joint torque produced by the muscles are: {muscular_joint_torque(act, q, qdot)}")

In [ ]:
# Now that we can compute the joint torque from the muscle the objective of our optimization will be to 
# compare this value to the inverse dynamic one trying to find the optimal solution
# Our cost function is therefore the difference between them
def cost_function(x, q, qdot, tau):
    activations = x
    return sum( (muscular_joint_torque(activations, q, qdot) - tau) ** 2 )

# We are now ready to try our optimization on a single frame
frame = 50
initial_guess = numpy.zeros((n_muscles, ))
q = all_q[:, frame]
qdot = all_qdot[:, frame]
tau = all_tau[:, frame]
results = optimize.minimize(cost_function, initial_guess, args=(q, qdot, tau), bounds=((0.001, 0.999),) * n_muscles)

print(f"The optimal cost function at frame {frame} for a set of activation ({results.x}) is: {results.fun}")

## What is going on? (If you are using the full body)
The value for the previous cell would be $0$ if the value perfectly matches the actual model. 
However a very large value is observed. 
Why is that?

Basically, the muscles solely act on the arm (as they are arm muscles). 
So everything related to the free-floating base and the lower body can't be actuated by these muscle.
Moreover, the muscles are too weak to produce the movement. 
The most common reason is because of a poor modelling of the muscle path (bad via points).

In any cases, the common way to solve this is to add virtual joint torque that will simulate the missing muscles and/or the missing force of the muscles.

In [ ]:
# So let's redefine our cost function assuming the x is now composite of activation and virtual joint torque
def cost_function(x, q, qdot, tau, weight_activation, weight_tau):
    activation = x[:n_muscles]
    virtual_tau = x[n_muscles:]
    computed_tau = muscular_joint_torque(activation, q, qdot) + virtual_tau
    return weight_activation * sum( (computed_tau - tau) ** 2 ) + weight_tau * sum(virtual_tau ** 2 )

# And rerun the optimization
frame = 50
initial_guess = numpy.zeros((n_muscles + n_q, ))
q = all_q[:, frame]
qdot = all_qdot[:, frame]
tau = all_tau[:, frame]
bounds = ((0.001, 0.999),) * n_muscles + ((-1000, 1000),) * n_q
weight_activation = 1
weight_tau = 0.001

results = optimize.minimize(cost_function, initial_guess, args=(q, qdot, tau, weight_activation, weight_tau), bounds=bounds)
results_activation = results.x[:n_muscles]
results_tau = results.x[n_muscles:]
print(f"The optimal cost function at frame {frame}\n" +
      f"for a set of activation ({results_activation}),\n" +
      f"using the virtual tau ({results_tau})\n" + 
      f"is: {results.fun}.")
print(f"Difference between the tau and tau from muscle and virtual_tau is: {muscular_joint_torque(results_activation, q, qdot) + results_tau - tau} Nm")

In [ ]:
# Now we can perform this at each frame
activations = []
virtual_taus = []
bounds = ((0.001, 0.999),) * n_muscles + ((-1000, 1000),) * n_q
weight_activation = 1
weight_tau = 0.001

initial_guess = numpy.zeros((n_muscles + n_q, ))
for i, (q, qdot, tau) in enumerate(zip(all_q.transpose(), all_qdot.transpose(), all_tau.transpose())):
    if i % 10 == 0:
        # Print something each 10 frames so the user does not panic
        print(f"Optimizing the {i}th frame")
        
    results = optimize.minimize(cost_function, initial_guess, args=(q, qdot, tau, weight_activation, weight_tau), bounds=bounds)
    
    activations.append(results.x[:n_muscles])
    virtual_taus.append(results.x[n_muscles:])

    # use the previous solution as the next initial_guess
    initial_guess = results.x
    
# Convert the results in a more convenient data structure
activations = numpy.array(activations).transpose()
virtual_taus = numpy.array(virtual_taus).transpose()    
print("Static optimization done!")

In [ ]:
# Finally, let's graph the results

# For the activation of all the muscles
for i, muscle_name in enumerate(muscle_names):
    pyplot.figure()
    pyplot.title(f"Activation of {muscle_name}")
    pyplot.ylabel("Activation (\%)")
    pyplot.plot(time_vector, activations[i, :])
pyplot.xlabel("Time (s)")

# For all the virtual joint torques
for i, dof_name in enumerate(dof_names):
    pyplot.figure()
    pyplot.title(f"Residual joint torque of {dof_name}")
    pyplot.ylabel("Joint torque (Nm)")
    pyplot.plot(time_vector, virtual_taus[i, :])
pyplot.xlabel("Time (s)")